# QAACCOUNT 账户类

QA_Account() 是quantaxis的核心类, 其作用是一个可以使用规则兼容各种市场的账户类

1.3.0以后, QA_Account需要由组合来进行创建(推荐)

```python

user = QA.QA_user(username ='quantaxis', password = 'quantaxis')
portfotlio=user.new_portfolio('x1')
account = portfolio.new_account(args)
```

## 调用方式

In [1]:
import QUANTAXIS as QA
import uuid
user = QA.QA_User(username ='quantaxis', password = 'quantaxis')
portfolio=user.new_portfolio('x1')
account = portfolio.new_account(account_cookie='test')

 prortfolio with user_cookie  USER_QKoncetW  already exist!!


## 参数详解

In [2]:
# strategy_name=None,
# user_cookie=None,
# portfolio_cookie=None,
# account_cookie=None,
# market_type=MARKET_TYPE.STOCK_CN,
# frequence=FREQUENCE.DAY,
# broker=BROKER_TYPE.BACKETEST,
# init_hold={},
# init_cash=1000000,
# commission_coeff=0.00025,
# tax_coeff=0.001,
# margin_level={},
# allow_t0=False,
# allow_sellopen=False,
# allow_margin=False,
# running_environment=RUNNING_ENVIRONMENT.BACKETEST

## 基于规则实例化

基于不同市场的不同规则, 我们可以实例化不同的账户类

- 允许保证金交易:  allow_marigin = True

- 允许买入后当日卖出: allow_t0 = True

- 允许卖空开仓(裸卖空): allow_sellopen= True

### 股票普通账户

In [3]:
stock_account= portfolio.new_account(account_cookie =str(uuid.uuid4()),allow_t0=False,allow_margin=False,allow_sellopen=False,market_type=QA.MARKET_TYPE.STOCK_CN)

QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE


### 股票日内回转账户

In [4]:
stockt0_account =portfolio.new_account(account_cookie =str(uuid.uuid4()),allow_t0=False,allow_margin=False,allow_sellopen=False,market_type=QA.MARKET_TYPE.STOCK_CN,init_hold={'000002':500}, running_environment=QA.RUNNING_ENVIRONMENT.TZERO)

QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE


###   股票融资融券账户

In [5]:
rzrq_account = portfolio.new_account(account_cookie =str(uuid.uuid4()),allow_t0=False,allow_margin=True,allow_sellopen=True,market_type=QA.MARKET_TYPE.STOCK_CN)

QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE


###  期货账户

In [6]:
future_account = portfolio.new_account(account_cookie =str(uuid.uuid4()),allow_t0=True,allow_margin=True,allow_sellopen=True, market_type=QA.MARKET_TYPE.FUTURE_CN)

QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE


### 期权账户

In [7]:
option_account = portfolio.new_account(account_cookie =str(uuid.uuid4()),allow_t0=True,allow_margin=True,allow_sellopen=True, market_type=QA.MARKET_TYPE.OPTION_CN)

QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE


###  其他市场账户


In [8]:
xxx = portfolio.new_account(account_cookie =str(uuid.uuid4()),allow_t0=True,allow_margin=True,allow_sellopen=True, market_type=QA.MARKET_TYPE.CRYPTOCURRENCY)

QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE


## 账户的初始资金/初始仓位

默认账户是无仓位, 默认现金 1000000 RMB


In [9]:
stock_account.init_assets

{'cash': 1000000.0, 'hold': {}}

In [10]:
stock_account.init_cash

1000000.0

In [11]:
stock_account.init_hold

Series([], Name: amount, dtype: float64)

###   在实例化的时候初始化仓位信息

使用json/dict的格式初始化  

```python
# init_hold参数
init_hold={code1:amount1,code2:amount2}
```

实例化完 会显示在 account.hold中

[注意] 在t+1的账户中, 初始化仓位依然可以当日交易

In [12]:
stock_account= portfolio.new_account(account_cookie =str(uuid.uuid4()),init_hold={'000001':500}, allow_t0=False,allow_margin=False,allow_sellopen=False,running_environment=QA.MARKET_TYPE.STOCK_CN)

QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE


In [13]:
stock_account.init_assets

{'cash': 1000000.0, 'hold': {'000001': 500}}

In [14]:
stock_account.hold

code
000001    500
Name: amount, dtype: int64

In [15]:
stock_account.sell_available

code
000001    500
Name: amount, dtype: int64

### 在实例化的时候初始现金信息

```python
# init_cash 参数
init_cash= 200000
```


In [16]:
stock_account= portfolio.new_account(account_cookie =str(uuid.uuid4()),init_cash= 200000,init_hold={'000002':500}, allow_t0=False,allow_margin=False,allow_sellopen=False,running_environment=QA.MARKET_TYPE.STOCK_CN)

QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE


In [17]:
stock_account.init_assets

{'cash': 200000.0, 'hold': {'000002': 500}}

In [18]:
stock_account.init_cash

200000.0

In [19]:
stock_account.cash

[200000.0]

In [20]:
stock_account.cash_available

200000.0

### 在已经实例化的账户中修改现金/ 重置现金操作

此操作无法撤销

- 现金记录全部消除
- 账户的持仓不会消除



In [21]:
stock_account.reset_assets(init_cash=50000)

In [22]:
stock_account.init_assets

{'cash': 50000.0, 'hold': {'000002': 500}}

In [23]:
stock_account.cash

[50000.0]

In [24]:
stock_account.cash_available

50000.0

##  买入/卖出操作

有2种方式可以买入/卖出品种

1. send_order 接口 + Order类调用receive_deal 接口
2. receive_simpledeal 接口


其中 
- 1 是基于账户-发出订单- 订单成交回报的模式更新账户
- 2 是直接修改账户的模式, 由用户自行确认和保证成交


###  订单/更新账户的模式

在此我们只演示订单/成交, 至于order.trade怎么成交, 则是市场类/broker类的问题, 在此不去演示

#### 生成订单

In [25]:
order=stock_account.send_order(code='000001',amount=100,time='2019-01-19',
                         amount_model=QA.AMOUNT_MODEL.BY_AMOUNT,order_model=QA.ORDER_MODEL.CLOSE,
                         price=10,towards=QA.ORDER_DIRECTION.BUY)

In [26]:
future_order=future_account.send_order(code='RB1905',amount=100,time='2019-01-19',
                         amount_model=QA.AMOUNT_MODEL.BY_AMOUNT,order_model=QA.ORDER_MODEL.CLOSE,
                         price=3500,towards=QA.ORDER_DIRECTION.BUY_OPEN)

In [27]:
future_account

< QA_Account 93ba32ae-6544-4aa1-ad92-b41f76365bc6 market: future_cn>

#### 订单类 QA_Order

In [28]:
order

< QA_Order realorder_id Order_BM1rYubU datetime:2019-01-19 09:31:00 code:000001 amount:100.0 price:10.0 towards:1 btype:stock_cn order_id:Order_BM1rYubU account:0dd507d7-5d1a-4140-9d32-d549b0f42f95 status:queued >

In [29]:
future_order

< QA_Order realorder_id Order_OAcNHXbT datetime:2019-01-19 09:31:00 code:RB1905 amount:100.0 price:3500.0 towards:2 btype:future_cn order_id:Order_OAcNHXbT account:93ba32ae-6544-4aa1-ad92-b41f76365bc6 status:queued >

#### 订单交易(成交在Market/Broker中判断,此处只演示成交回报)

In [30]:
order.trade(trade_price=10.1,trade_amount=100,trade_id='example_trade1',trade_time='2019-01-19 15:00:00')

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!receive deal


In [31]:
future_order.trade(trade_price=3600,trade_amount=100,trade_id='example_trade2',trade_time='2019-01-19 21:00:00')

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!receive deal


#### 账户成交表  QA_Account.history/QA_Account.history_table

In [32]:
stock_account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax,message,frozen,direction
0,2019-01-19 15:00:00,000001,10.1,100,48985.0,Order_BM1rYubU,Order_BM1rYubU,example_trade1,0dd507d7-5d1a-4140-9d32-d549b0f42f95,5.0,0.0,None,0,1


In [33]:
future_account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax,message,frozen,direction
0,2019-01-19 21:00:00,RB1905,3600.0,100,675640.0,Order_OAcNHXbT,Order_OAcNHXbT,example_trade2,93ba32ae-6544-4aa1-ad92-b41f76365bc6,360.0,0.0,None,324000.0,2


#### 账户现金表/ 期货账户还会冻结资金(保证金)

In [34]:
stock_account.cash

[50000.0, 48985.0]

In [35]:
future_account.cash

[1000000.0, 675640.0]

In [36]:
future_account.frozen

{'RB1905': {'2': {'money': 3240.0, 'amount': 100, 'avg_price': 3600.0},
  '-2': {'money': 0, 'amount': 0, 'avg_price': 0}}}

#### 账户可用现金

In [37]:
stock_account.cash_available

48985.0

In [38]:
future_account.cash_available

675640.0

#### 账户持仓

In [39]:
stock_account.hold

code
000001    100
000002    500
Name: amount, dtype: int64

In [40]:
future_account.hold

code
RB1905    100
Name: amount, dtype: int64

#### 账户可卖余额(股票T+1 因此可卖部分不增加/ 期货账户 t+0 因此可卖部分增加)

In [41]:
stock_account.sell_available

code
000002    500
Name: amount, dtype: int64

In [42]:
future_account.sell_available

code
RB1905    100
Name: amount, dtype: int64

###    直接操作账户模式

该模式直接操作账户

In [43]:
stock_account.receive_simpledeal(code='000004',order_id='model2',realorder_id='model2_real',trade_id='trade2',trade_amount=1000,trade_price=16,trade_time='2019-01-21',trade_towards=QA.ORDER_DIRECTION.BUY,message='model2')

In [44]:
stock_account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax,message,frozen,direction
0,2019-01-19 15:00:00,000001,10.1,100,48985.0,Order_BM1rYubU,Order_BM1rYubU,example_trade1,0dd507d7-5d1a-4140-9d32-d549b0f42f95,5.0,0.0,None,0,1
1,2019-01-21,000004,16.0,1000,32980.0,model2,model2_real,trade2,0dd507d7-5d1a-4140-9d32-d549b0f42f95,5.0,0.0,model2,0,1


In [45]:
stock_account.cash

[50000.0, 48985.0, 32980.0]

In [46]:
stock_account.cash_available

32980.0

In [47]:
stock_account.hold

code
000001     100
000002     500
000004    1000
Name: amount, dtype: int64

In [48]:
stock_account.sell_available

code
000002    500
Name: amount, dtype: int64

##   结算

主要是股票账户的可卖股数的结算

目前期货账户不采用逐日盯市的结算方式(及无结算价/无当日盈亏)

In [49]:
stock_account.settle()

In [50]:
stock_account.sell_available

code
000001     100
000002     500
000004    1000
Name: amount, dtype: int64

In [51]:
future_account.settle()

In [52]:
##  关于账户存储和恢复

## 动态字段

In [53]:
future_account

< QA_Account 93ba32ae-6544-4aa1-ad92-b41f76365bc6 market: future_cn>

In [54]:
future_account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax,message,frozen,direction
0,2019-01-19 21:00:00,RB1905,3600.0,100,675640.0,Order_OAcNHXbT,Order_OAcNHXbT,example_trade2,93ba32ae-6544-4aa1-ad92-b41f76365bc6,360.0,0.0,None,324000.0,2


In [55]:
future_account.frozen

{'RB1905': {'2': {'money': 3240.0, 'amount': 100, 'avg_price': 3600.0},
  '-2': {'money': 0, 'amount': 0, 'avg_price': 0}}}

In [56]:
future_account.cash_available

675640.0

In [57]:
future_account.sell_available

code
RB1905    100
Name: amount, dtype: int64

In [58]:
future_account.hold_available

code
RB1905    100
Name: amount, dtype: int64

In [59]:
stock_account

< QA_Account 0dd507d7-5d1a-4140-9d32-d549b0f42f95 market: stock_cn>

In [60]:
stockt0_account.hold_available

Series([], Name: amount, dtype: float64)

In [61]:
stockt0_account.sell_available

code
000002    500
Name: amount, dtype: int64

In [62]:
stockt0_account.buy_available

code
000002    500
Name: amount, dtype: int64

In [63]:
ordert0=stockt0_account.send_order(code='000002',amount=100,time='2019-01-19',
                         amount_model=QA.AMOUNT_MODEL.BY_AMOUNT,order_model=QA.ORDER_MODEL.CLOSE,
                         price=10,towards=QA.ORDER_DIRECTION.BUY)

In [64]:
stockt0_account.sell_available

code
000002    500
Name: amount, dtype: int64

In [65]:
stockt0_account.buy_available

code
000002    400
Name: amount, dtype: int64

In [66]:
stockt0_account.hold_available

Series([], Name: amount, dtype: float64)

In [67]:
ordert0.trade(trade_price=10.1,trade_amount=100,trade_id='example_trade1',trade_time='2019-01-19 15:00:00')

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!receive deal


In [68]:
stockt0_account.hold_available

code
000002    100
Name: amount, dtype: int64

In [69]:
stockt0_account.sell_available

code
000002    500
Name: amount, dtype: int64

In [70]:
stockt0_account.buy_available

code
000002    400
Name: amount, dtype: int64